In [1]:
import paddle
print(paddle.__version__)

grep: warning: GREP_OPTIONS is deprecated; please use an alias or script


0.0.0


In [2]:
from paddle.jit import to_static
from paddle.static import InputSpec
paddle.set_device('cpu')

################## 模型组网 ##################
x_spec = [InputSpec([None, 4],dtype='float32')]
class Net(paddle.nn.Layer):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = paddle.nn.Linear(4, 4)
        self.fc2 = paddle.nn.Linear(4, 4)
        self.bias = 0.4
        self.flag = paddle.ones([2],dtype="int32")
    ################## 导出函数 ① ##################
    @to_static(input_spec=x_spec)
    def forward(self, x):
        out = self.fc1(x)
        out = paddle.nn.functional.relu(out)
        out = paddle.mean(out)
        return out
    ################## 导出函数 ② ##################
    @to_static(input_spec=x_spec)
    def infer(self, input):
        out = self.fc2(input)
        out = out + self.bias
        out = paddle.mean(out)
        return out
    ################## 导出变量 ① ##################
    # For extra Python float
    # @to_static(property=True)
    def bias(self):
        return self.bias + 1
    ################## 导出变量 ② ##################
    # For extra Tensor
    # @to_static(property=True)
    def flag(self):
        return self.flag
    
################## 模型导出 ##################
net = Net()

In [4]:
# getattr will trick decarator
for i in dir(net):
    print(i, type(getattr(net, i, None)))

__call__ <class 'method'>
__class__ <class 'type'>
__delattr__ <class 'method'>
__dict__ <class 'dict'>
__dir__ <class 'method'>
__doc__ <class 'NoneType'>
__eq__ <class 'method-wrapper'>
__format__ <class 'builtin_function_or_method'>
__ge__ <class 'method-wrapper'>
__getattr__ <class 'method'>
__getattribute__ <class 'method-wrapper'>
__getstate__ <class 'method'>
__gt__ <class 'method-wrapper'>
__hash__ <class 'method-wrapper'>
__init__ <class 'method'>
__init_subclass__ <class 'builtin_function_or_method'>
__le__ <class 'method-wrapper'>
__lt__ <class 'method-wrapper'>
__module__ <class 'str'>
__ne__ <class 'method-wrapper'>
__new__ <class 'builtin_function_or_method'>
__reduce__ <class 'builtin_function_or_method'>
__reduce_ex__ <class 'builtin_function_or_method'>
__repr__ <class 'method'>
__setattr__ <class 'method'>
__setstate__ <class 'method'>
__sizeof__ <class 'builtin_function_or_method'>
__str__ <class 'method-wrapper'>
__subclasshook__ <class 'builtin_function_or_method'>

In [13]:
d = net.to_static_state_dict()
print(d.keys())

odict_keys(['flag', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])


In [17]:
s = paddle.fluid.core.Scope()
dir(s)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_kids',
 '_remove_from_pool',
 'drop_kids',
 'erase',
 'find_var',
 'new_scope',
 'size',
 'var']

In [20]:
s.find_var?

Docstring:
find_var(self: paddle.fluid.core_avx._Scope, name: str) -> paddle.fluid.core_avx.Variable


Find variable named :code:`name` in the current scope or
its parent scope. Return None if not found. 

Args:
    name (str): the variable name.

Returns:
    out (core.Variable|None): the found variable or None.
Type:      method


In [21]:
s.var?

Docstring:
var(self: paddle.fluid.core_avx._Scope, name: str) -> paddle.fluid.core_avx.Variable


Find or create variable named :code:`name` in the current scope.

If the variable named :code:`name` does not exist in the
current scope, the variable would be created. Otherwise,
return the existing variable.

Args:
    name (str): the variable name.

Returns:
    out (core.Variable): the found or created variable.
Type:      method


In [23]:
s.size?

Docstring: size(self: paddle.fluid.core_avx._Scope) -> int
Type:      method


In [3]:

# set "use_combine=True" to save for Layer
# export.xxx, export.infer.xxx
# export.extra -- for all attribute
#   - {'bias': self.bias, 'flag': self.flag}
paddle.jit.save(net, path="./export", use_combine=True)

{var mean_0.tmp_0 : LOD_TENSOR.shape(1,).dtype(float32).stop_gradient(False), var linear_2.tmp_1 : LOD_TENSOR.shape(-1, 4).dtype(float32).stop_gradient(False), persist trainable param linear_0.w_0 : LOD_TENSOR.shape(4, 4).dtype(float32).stop_gradient(False), var relu_0.tmp_0 : LOD_TENSOR.shape(-1, 4).dtype(float32).stop_gradient(False), persist trainable param linear_0.b_0 : LOD_TENSOR.shape(4,).dtype(float32).stop_gradient(False), var x : LOD_TENSOR.shape(-1, 4).dtype(float32).stop_gradient(False), persist trainable param linear_1.w_0 : LOD_TENSOR.shape(4, 4).dtype(float32).stop_gradient(False), var linear_3.tmp_0 : LOD_TENSOR.shape(-1, 4).dtype(float32).stop_gradient(False), persist var generated_tensor_0 : LOD_TENSOR.shape(2,).dtype(int32).stop_gradient(True), var x : LOD_TENSOR.shape(-1, 4).dtype(float32).stop_gradient(False), var tmp_0 : LOD_TENSOR.shape(-1, 4).dtype(float32).stop_gradient(False), var mean_1.tmp_0 : LOD_TENSOR.shape(1,).dtype(float32).stop_gradient(False), var lin

/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/paddle/fluid/io.py:1444: UserWarning: save_inference_model specified the param `program_only` to True, It will not save params of Program.
  "save_inference_model specified the param `program_only` to True, It will not save params of Program."


In [4]:
################## 模型ji加载 ##################
# set "use_combine=True" to load as jit::Layer
new_net = paddle.jit.load(path="./export", use_combine=True)
################## 函数前后端灵活调用 ##################
x = paddle.randn([2,4])
# C++端： jit::Layer.Forward(x)
out = new_net(x)
# C++端： jit::Layer.GetMethod("infer")(x)
pred = new_net.infer(x)
################## 变量前后端灵活访问 ##################
# C++端： jit::Layer.GetAttribute("bias")
bias = new_net.bias()
# C++端： jit::Layer.GetAttribute("flags")
flag = new_net.flags()

ValueError: The additional config (use_combine) of `paddle.jit.load` is not supported.

In [8]:
import paddle
from paddle.jit import to_static
from paddle.static import InputSpec
paddle.set_device('cpu')

from functools import wraps

def wrapper(method):
    print(dir(method))
    def _impl(self, *method_args, **method_kwargs):
        method._origin = method
        method_output = method(self, *method_args, **method_kwargs)
        return method_output + "!"
    return _impl

   
################## 模型组网 ##################
x_spec = [InputSpec([None, 4],dtype='float32')]
class Net(paddle.nn.Layer):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = paddle.nn.Linear(4, 4)
        self.fc2 = paddle.nn.Linear(4, 4)
        self.bias = 0.4
        self.flag = paddle.ones([2],dtype="int32")
    
    def test(self):
        return 'a'
    
    @wrapper
    def bar(self, word):
        return word
    
    ################## 导出函数 ① ##################
    @to_static(input_spec=x_spec)
    def forward(self, x):
        out = self.fc1(x)
        out = paddle.nn.functional.relu(out)
        out = paddle.mean(out)
        return out
    ################## 导出函数 ② ##################
    @to_static(input_spec=x_spec)
    def infer(self, input):
        out = self.fc2(input)
        out = out + self.bias
        out = paddle.mean(out)
        return out
    
    @to_static(input_spec=x_spec)
    def log_softmax(self, input):
        return paddle.nn.functional.log_softmax(input, axis=-1)
    
    ################## 导出变量 ① ##################
    # For extra Python float
    @to_static(property=True)
    def fbias(self):
        return self.bias + 1
    ################## 导出变量 ② ##################
    # For extra Tensor
    @to_static(property=True)
    def fflag(self):
        return self.flag
    
################## 模型导出 ##################
net = Net()

['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']
<function Net.forward at 0x7fc23e7d2d08>
['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']
(self, 

In [2]:
import inspect
print(inspect.ismethod(net.flag))
print(inspect.ismethod(net.test))
print(inspect.ismethod(net.bar))

not False <function Net.flag at 0x7fc23e937e18>
False
True
True


In [46]:
print(net.bar)
print(net.flag)
print(net.flag._dygraph_function)
print(inspect.ismethod(net.flag._dygraph_function))
print(inspect.signature(net.flag._dygraph_function))

<bound method wrapper.<locals>._impl of Net(
  (fc1): Linear(in_features=4, out_features=4, dtype=float32)
  (fc2): Linear(in_features=4, out_features=4, dtype=float32)
)>
<function Net.flag at 0x7f0adf9b0158>
False
(self)


In [18]:
def a():
    return 1

print(inspect.ismethod(a))

False


In [20]:
inspect.getmembers(net.flag)

[('__call__', <method-wrapper '__call__' of method object at 0x7f0adfc23248>),
 ('__class__', method),
 ('__delattr__',
  <method-wrapper '__delattr__' of method object at 0x7f0adfc23248>),
 ('__dir__', <function method.__dir__()>),
 ('__doc__', None),
 ('__eq__', <method-wrapper '__eq__' of method object at 0x7f0adfc23248>),
 ('__format__', <function method.__format__(format_spec, /)>),
 ('__func__', <function __main__.Net.flag(self)>),
 ('__ge__', <method-wrapper '__ge__' of method object at 0x7f0adfc23248>),
 ('__get__', <method-wrapper '__get__' of method object at 0x7f0adfc23248>),
 ('__getattribute__',
  <method-wrapper '__getattribute__' of method object at 0x7f0adfc23248>),
 ('__gt__', <method-wrapper '__gt__' of method object at 0x7f0adfc23248>),
 ('__hash__', <method-wrapper '__hash__' of method object at 0x7f0adfc23248>),
 ('__init__', <method-wrapper '__init__' of method object at 0x7f0adfc23248>),
 ('__init_subclass__', <function method.__init_subclass__>),
 ('__le__', <me

In [4]:
# getattr will trick decarator
for i in dir(net):
    if i == 'fbias':
        print(i, type(getattr(net, i, None)))
        a = getattr(net, i)
        print(a._kwargs)
        print(net.fbias._kwargs)
        print(net.fbias.is_property)

    if i == 'flag':
        print(i, type(getattr(net, i, None)))
        a = getattr(net, i)
        print(a._kwargs)
        
    if i == 'infer':
        print(i, type(getattr(net, i, None)))
        a = getattr(net, i)
        print(a._kwargs)

fbias <class 'paddle.fluid.dygraph.dygraph_to_static.program_translator.StaticFunction'>
{'build_strategy': <paddle.fluid.core_avx.ParallelExecutor.BuildStrategy object at 0x7fc23e934a40>, 'property': True}
{'build_strategy': <paddle.fluid.core_avx.ParallelExecutor.BuildStrategy object at 0x7fc23e934a40>, 'property': True}
True
flag <class 'paddle.fluid.dygraph.dygraph_to_static.program_translator.StaticFunction'>
{'build_strategy': <paddle.fluid.core_avx.ParallelExecutor.BuildStrategy object at 0x7fc23e934b90>, 'property': True}
flag <class 'paddle.fluid.dygraph.dygraph_to_static.program_translator.StaticFunction'>
{'build_strategy': <paddle.fluid.core_avx.ParallelExecutor.BuildStrategy object at 0x7fc23e934b90>, 'property': True}
infer <class 'paddle.fluid.dygraph.dygraph_to_static.program_translator.StaticFunction'>
{'build_strategy': <paddle.fluid.core_avx.ParallelExecutor.BuildStrategy object at 0x7fc23e934928>, 'property': False}


In [9]:
paddle.jit.save(net, path="./export_p", use_combine=True)

not False <function Net.fbias at 0x7fc23e7bc950>
not False <function Net.fflag at 0x7fc23e7bd2f0>
not False <function Net.forward at 0x7fc23e7d2d08>
not False <function Net.infer at 0x7fc23e7c96a8>
not False <function Net.log_softmax at 0x7fc23e7bc048>
